In [1]:
#libraries

import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files
import sqlalchemy
import os

In [25]:
agri_land = pd.read_csv("kaggle/Agricultural Land.csv", index_col=0)
corps_units = pd.read_csv("kaggle/Crops_Units.csv", index_col=0)
fresh_gw = pd.read_csv("kaggle/Fresh groundwater abstracted.csv", index_col=0)
fresh_ab = pd.read_csv("kaggle/Freshwater abstracted.csv", index_col=0)
ghg = pd.read_csv("kaggle/GHG_Emissions_by_Sector.csv", index_col=0)
prec = pd.read_csv("Precipitation.csv", index_col=0)

In [26]:
prec

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes\n2008,Footnotes\n2009,Footnotes\n2010,Footnotes\n2011,Footnotes\n2012,Footnotes\n2013,Footnotes\n2014,Footnotes\n2015,Footnotes\n2016,Footnotes\n2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,28385,...,...,...,...,40311,...,...,...,...,1.0,1.0,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0
12.0,Algeria,275681.3125,...,...,...,...,376971.5,194723.2969,217926,198905.2031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20.0,Andorra,471.7792969,...,...,...,...,424.8518982,625.1678467,444.7502136,361.2449341,...,3.0,3.0,3,3.0,3.0,4.0,4.0,4.0,NaN,NaN
660.0,Anguilla,93.09999847,...,...,...,...,100.7300034,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28.0,Antigua and Barbuda,300.2999878,264.3999939,475.7999878,322.5,283.7999878,374.5,323.2999878,279.2000122,384.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,68000,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,244262.2031,...,...,...,...,163649.0313,273803.4375,315223.6875,209523.9063,...,34.0,34.0,34,34.0,34.0,34.0,34.0,34.0,34.0,34.0


In [27]:
prec.isnull().sum()

Country              2
1990                 2
1991                 2
1992                 2
1993                 2
1994                 2
1995                 2
1996                 2
1997                 2
1998                 2
1999                 2
2000                 2
2001                 2
2002                 2
2003                 2
2004                 2
2005                 2
2006                 2
2007                 2
2008                 2
2009                 2
2010                 2
2011                 2
2012                 2
2013                 2
2014                 2
2015                 2
2016                 2
2017                 2
Footnotes\n1990    111
Footnotes\n1991    120
Footnotes\n1992    120
Footnotes\n1993    120
Footnotes\n1994    120
Footnotes\n1995    113
Footnotes\n1996    112
Footnotes\n1997    111
Footnotes\n1998    110
Footnotes\n1999    109
Footnotes\n2000    109
Footnotes\n2001    109
Footnotes\n2002    109
Footnotes\n2003    110
Footnotes\n

In [28]:
prec.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes\n1990', 'Footnotes\n1991',
       'Footnotes\n1992', 'Footnotes\n1993', 'Footnotes\n1994',
       'Footnotes\n1995', 'Footnotes\n1996', 'Footnotes\n1997',
       'Footnotes\n1998', 'Footnotes\n1999', 'Footnotes\n2000',
       'Footnotes\n2001', 'Footnotes\n2002', 'Footnotes\n2003',
       'Footnotes\n2004', 'Footnotes\n2005', 'Footnotes\n2006',
       'Footnotes\n2007', 'Footnotes\n2008', 'Footnotes\n2009',
       'Footnotes\n2010', 'Footnotes\n2011', 'Footnotes\n2012',
       'Footnotes\n2013', 'Footnotes\n2014', 'Footnotes\n2015',
       'Footnotes\n2016', 'Footnotes\n2017'],
      dtype='object')

In [29]:
prec.drop(['Footnotes\n1990', 'Footnotes\n1991',
       'Footnotes\n1992', 'Footnotes\n1993', 'Footnotes\n1994',
       'Footnotes\n1995', 'Footnotes\n1996', 'Footnotes\n1997',
       'Footnotes\n1998', 'Footnotes\n1999', 'Footnotes\n2000',
       'Footnotes\n2001', 'Footnotes\n2002', 'Footnotes\n2003',
       'Footnotes\n2004', 'Footnotes\n2005', 'Footnotes\n2006',
       'Footnotes\n2007', 'Footnotes\n2008', 'Footnotes\n2009',
       'Footnotes\n2010', 'Footnotes\n2011', 'Footnotes\n2012',
       'Footnotes\n2013', 'Footnotes\n2014', 'Footnotes\n2015',
       'Footnotes\n2016', 'Footnotes\n2017'], axis='columns',
                                        inplace= True)

In [30]:
prec

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,28385,...,...,...,...,40311,...,...,...,...,29458,37218,51134,20569,43050,30615,37172,32711,38810,32224
12.0,Algeria,275681.3125,...,...,...,...,376971.5,194723.2969,217926,198905.2031,...,544728.625,313696.5,373310.0938,291255.5,285533.0938,276258.9063,237403,238029.5,181922.0938,330957.4063
20.0,Andorra,471.7792969,...,...,...,...,424.8518982,625.1678467,444.7502136,361.2449341,...,466.674469,464.0131226,423.6300049,355.0338745,352.9525146,482.6208496,551.6988525,474.7380066,...,...
660.0,Anguilla,93.09999847,...,...,...,...,100.7300034,...,...,...,...,96.88999939,71.08000183,...,...,...,...,...,...,...,...
28.0,Antigua and Barbuda,300.2999878,264.3999939,475.7999878,322.5,283.7999878,374.5,323.2999878,279.2000122,384.5,...,392.5,276.8999939,463.8999939,450.2000122,279.7000122,328.2000122,257.5,161.1000061,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,68000,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894.0,Zambia,...,...,...,...,...,...,...,...,...,...,1019.900024,1002.200012,1008.799988,890.4000244,1033.900024,...,...,...,...,...
716.0,Zimbabwe,244262.2031,...,...,...,...,163649.0313,273803.4375,315223.6875,209523.9063,...,311128.875,286966.0313,277085.7813,291270.2813,259853.4063,300531.2188,294826.1563,244809.2656,211907.5156,372899.4063


In [4]:
agri_land

,Agricultural area in 2013 (km2),% change of agricultural area since 1990,% of total land area covered by agricultural area in 2013,Arable land in 2013 (km2),Permanent crops in 2013 (km2),Permanent meadows and pastures in 2013 (km2),Agricultural area actually irrigated in 2013 (km2),Unnamed: 8
Country,,,,,,,,
Afghanistan,379 100,-0.3,58.1,77 850,1 250,300 000,20 920,NaN
Albania,11 873,5.9,43.3,6 171,792,4 910,2 053,NaN
Algeria,414 316,7.1,17.4,74 962,9 390,329 964,10 895,NaN
American Samoa,49,63.3,24.5,30,19,...,...,NaN
Andorra,208,9.5,44.3,29,...,179,...,NaN
...,...,...,...,...,...,...,...,...
Wallis and Futuna Islands,60,0.0,42.9,10,50,...,...,NaN
Western Sahara,50 040,0.0,18.8,40,...,50 000,...,NaN
Yemen,235 460,-0.3,44.6,12 480,2 980,220 000,...,NaN


In [31]:
agri_land.columns

Index(['Agricultural area in 2013 (km2)',
       '% change of agricultural area since 1990',
       '% of total land area covered by agricultural area in 2013',
       'Arable land in 2013 (km2)', 'Permanent crops in 2013 (km2)',
       'Permanent meadows and pastures in 2013 (km2)',
       'Agricultural area actually irrigated in 2013 (km2)', 'Unnamed: 8'],
      dtype='object')

In [21]:
agri_land.isnull().sum()

Agricultural area in 2013 (km2)                                0
% change of agricultural area since 1990                       0
% of total land area covered by agricultural area in 2013      0
Arable land in 2013 (km2)                                      0
Permanent crops in 2013 (km2)                                  0
Permanent meadows and pastures in 2013 (km2)                   0
Agricultural area actually irrigated in 2013 (km2)             0
Unnamed: 8                                                   225
dtype: int64

In [32]:
agri_land.drop('Unnamed: 8', axis='columns', inplace= True)

In [33]:
agri_land

,Agricultural area in 2013 (km2),% change of agricultural area since 1990,% of total land area covered by agricultural area in 2013,Arable land in 2013 (km2),Permanent crops in 2013 (km2),Permanent meadows and pastures in 2013 (km2),Agricultural area actually irrigated in 2013 (km2)
Country,,,,,,,
Afghanistan,379 100,-0.3,58.1,77 850,1 250,300 000,20 920
Albania,11 873,5.9,43.3,6 171,792,4 910,2 053
Algeria,414 316,7.1,17.4,74 962,9 390,329 964,10 895
American Samoa,49,63.3,24.5,30,19,...,...
Andorra,208,9.5,44.3,29,...,179,...
...,...,...,...,...,...,...,...
Wallis and Futuna Islands,60,0.0,42.9,10,50,...,...
Western Sahara,50 040,0.0,18.8,40,...,50 000,...
Yemen,235 460,-0.3,44.6,12 480,2 980,220 000,...


In [5]:
agri_land.agg('Agricultural area in 2013 (km2)')

Country
Afghanistan                    379 100
Albania                         11 873
Algeria                        414 316
American Samoa                      49
Andorra                            208
                               ...    
Wallis and Futuna Islands           60
Western Sahara                  50 040
Yemen                          235 460
Zambia                         237 360
Zimbabwe                       162 000
Name: Agricultural area in 2013 (km2), Length: 225, dtype: object

In [6]:
corps_units.describe()

,Description
count,60
unique,59
top,United States dollar
freq,2


In [7]:
fresh_gw

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes 2008,Footnotes 2009,Footnotes 2010,Footnotes 2011,Footnotes 2012,Footnotes 2013,Footnotes 2014,Footnotes 2015,Footnotes 2016,Footnotes 2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,2,2.0,2.0,2.0,2.0
28.0,Antigua and Barbuda,1.139999986,...,...,...,...,0.550000012,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51.0,Armenia,1325.400024,...,...,...,...,851,616,520,530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31.0,Azerbaijan,1706,...,...,...,...,1150,987,1098,681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,2442,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,286.5,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,9270.139648,...,...,...,...,7572.710449,10668.95801,11732.38574,10895.43945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
fresh_gw.isnull().sum()

Country             2
1990                2
1991                2
1992                2
1993                2
1994                2
1995                2
1996                2
1997                2
1998                2
1999                2
2000                2
2001                2
2002                2
2003                2
2004                2
2005                2
2006                2
2007                2
2008                2
2009                2
2010                2
2011                2
2012                2
2013                2
2014                2
2015                2
2016                2
2017                2
Footnotes 1990    109
Footnotes 1991    114
Footnotes 1992    114
Footnotes 1993    114
Footnotes 1994    114
Footnotes 1995    110
Footnotes 1996    112
Footnotes 1997    112
Footnotes 1998    112
Footnotes 1999    110
Footnotes 2000    108
Footnotes 2001    106
Footnotes 2002    108
Footnotes 2003    108
Footnotes 2004    105
Footnotes 2005     97
Footnotes 

In [35]:
fresh_gw.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes 1990', 'Footnotes 1991',
       'Footnotes 1992', 'Footnotes 1993', 'Footnotes 1994', 'Footnotes 1995',
       'Footnotes 1996', 'Footnotes 1997', 'Footnotes 1998', 'Footnotes 1999',
       'Footnotes 2000', 'Footnotes 2001', 'Footnotes 2002', 'Footnotes 2003',
       'Footnotes 2004', 'Footnotes 2005', 'Footnotes 2006', 'Footnotes 2007',
       'Footnotes 2008', 'Footnotes 2009', 'Footnotes 2010', 'Footnotes 2011',
       'Footnotes 2012', 'Footnotes 2013', 'Footnotes 2014', 'Footnotes 2015',
       'Footnotes 2016', 'Footnotes 2017'],
      dtype='object')

In [38]:
fresh_gw.drop(['Footnotes 1990', 'Footnotes 1991',
       'Footnotes 1992', 'Footnotes 1993', 'Footnotes 1994', 'Footnotes 1995',
       'Footnotes 1996', 'Footnotes 1997', 'Footnotes 1998', 'Footnotes 1999',
       'Footnotes 2000', 'Footnotes 2001', 'Footnotes 2002', 'Footnotes 2003',
       'Footnotes 2004', 'Footnotes 2005', 'Footnotes 2006', 'Footnotes 2007',
       'Footnotes 2008', 'Footnotes 2009', 'Footnotes 2010', 'Footnotes 2011',
       'Footnotes 2012', 'Footnotes 2013', 'Footnotes 2014', 'Footnotes 2015',
       'Footnotes 2016', 'Footnotes 2017'], axis='columns',
                                        inplace= True)

In [39]:
fresh_gw

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,235,262,...,107
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,4150,4200,2603,2915,3000,3046.909912,3117.449951,3654.939941,3900,3860
28.0,Antigua and Barbuda,1.139999986,...,...,...,...,0.550000012,...,...,...,...,1.399999976,1.600000024,1.600000024,1.200000048,1.200000048,0.720000029,0.680000007,0.74000001,...,...
51.0,Armenia,1325.400024,...,...,...,...,851,616,520,530,...,852,854.4000244,875.7999878,1002.799988,1314.400024,1348.699951,1312,1304.400024,1136.300049,1154.5
31.0,Azerbaijan,1706,...,...,...,...,1150,987,1098,681,...,1127,1262,1272,1506,1696,1738,1819,2016,2075,2066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,2442,...,...,...,...,...,...,3805,...,...,...,...,4411,...,...
894.0,Zambia,...,...,...,...,...,286.5,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716.0,Zimbabwe,9270.139648,...,...,...,...,7572.710449,10668.95801,11732.38574,10895.43945,...,14684.69141,16608.80664,16810.83594,16906.18359,13721.11328,14798.3916,13818.21973,10370.45215,12507.52441,13714.00684


In [8]:
fresh_ab

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,Footnotes\r\n2008,Footnotes\r\n2009,Footnotes\r\n2010,Footnotes\r\n2011,Footnotes\r\n2012,Footnotes\r\n2013,Footnotes\r\n2014,Footnotes\r\n2015,Footnotes\r\n2016,Footnotes\r\n2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20.0,Andorra,...,...,...,...,...,...,...,...,...,...,2,2,2.0,2.0,2.0,2,2.0,2.0,NaN,NaN
28.0,Antigua and Barbuda,3.380000114,...,...,...,...,0.970000029,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51.0,Armenia,3942,...,...,...,...,2331,2077,2250.699951,2144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,3942,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
894.0,Zambia,...,...,...,...,...,2431,...,...,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716.0,Zimbabwe,33107.64063,...,...,...,...,27045.39453,38103.42188,41901.375,38912.28516,...,39,39,39.0,39.0,39.0,39,39.0,39.0,39.0,39.0


In [41]:
fresh_ab.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', 'Footnotes\r\n1990', 'Footnotes\r\n1991',
       'Footnotes\r\n1992', 'Footnotes\r\n1993', 'Footnotes\r\n1994',
       'Footnotes\r\n1995', 'Footnotes\r\n1996', 'Footnotes\r\n1997',
       'Footnotes\r\n1998', 'Footnotes\r\n1999', 'Footnotes\r\n2000',
       'Footnotes\r\n2001', 'Footnotes\r\n2002', 'Footnotes\r\n2003',
       'Footnotes\r\n2004', 'Footnotes\r\n2005', 'Footnotes\r\n2006',
       'Footnotes\r\n2007', 'Footnotes\r\n2008', 'Footnotes\r\n2009',
       'Footnotes\r\n2010', 'Footnotes\r\n2011', 'Footnotes\r\n2012',
       'Footnotes\r\n2013', 'Footnotes\r\n2014', 'Footnotes\r\n2015',
       'Footnotes\r\n2016', 'Footnotes\r\n2017'],
      dtype='object')

In [43]:
fresh_ab.drop(['Footnotes\r\n1990', 'Footnotes\r\n1991',
       'Footnotes\r\n1992', 'Footnotes\r\n1993', 'Footnotes\r\n1994',
       'Footnotes\r\n1995', 'Footnotes\r\n1996', 'Footnotes\r\n1997',
       'Footnotes\r\n1998', 'Footnotes\r\n1999', 'Footnotes\r\n2000',
       'Footnotes\r\n2001', 'Footnotes\r\n2002', 'Footnotes\r\n2003',
       'Footnotes\r\n2004', 'Footnotes\r\n2005', 'Footnotes\r\n2006',
       'Footnotes\r\n2007', 'Footnotes\r\n2008', 'Footnotes\r\n2009',
       'Footnotes\r\n2010', 'Footnotes\r\n2011', 'Footnotes\r\n2012',
       'Footnotes\r\n2013', 'Footnotes\r\n2014', 'Footnotes\r\n2015',
       'Footnotes\r\n2016', 'Footnotes\r\n2017'], axis='columns',
                                        inplace= True)

In [44]:
fresh_ab

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
CountryID,,,,,,,,,,,,,,,,,,,,,
8.0,Albania,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,1123,1194,...,1188
12.0,Algeria,...,...,...,...,...,...,...,...,...,...,6950,7150,7203,7515,7800,4642.759766,4819.819824,5512.660156,7800,7730
20.0,Andorra,...,...,...,...,...,...,...,...,...,...,16.39779282,15.4166193,15.82946682,16.04419327,16.58762169,17.15185165,15.44569111,15.66994095,...,...
28.0,Antigua and Barbuda,3.380000114,...,...,...,...,0.970000029,...,...,...,...,3.529999971,3.950000048,3.789999962,3.75999999,3.809999943,2.769999981,1.080000043,1.25,...,...
51.0,Armenia,3942,...,...,...,...,2331,2077,2250.699951,2144,...,2873.5,2504.699951,2326.399902,2438.300049,2941.199951,2955.100098,2860.199951,3271.699951,3181.899902,2865.399902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887.0,Yemen,...,...,...,...,...,3942,...,...,...,...,...,...,5305,...,...,...,...,5911,...,...
894.0,Zambia,...,...,...,...,...,2431,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716.0,Zimbabwe,33107.64063,...,...,...,...,27045.39453,38103.42188,41901.375,38912.28516,...,52445.32813,59317.16797,60038.69531,60379.23047,49003.97656,52851.39844,49350.78125,37037.32813,44669.73047,48978.59766


In [9]:
ghg

,Country,Latest Year Available,Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent),GHG from energy (1000 tonnes of CO₂ equivalent),"GHG from energy, as percentage to total","GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)","GHG from energy, of which: from transport, as percentage to total",GHG from industrial processes and product use (1000 tonnes of CO₂ equivalent),"GHG from industrial processes and product use, as percentage to total",GHG from agriculture (1000 tonnes of CO₂ equivalent),"GHG from agriculture, as percentage to total",GHG from waste (1000 tonnes of CO₂ equivalent),"GHG from waste, as percentage to total"
Country ID,,,,,,,,,,,,,
4,Afghanistan,2013,"43,377.00","10,281.00",23.7,"5,217.00",12.0,212.00,0.5,"32,744.00",75.5,140.00,0.3
8,Albania,2009,"8,125.70","4,466.04",55.0,"2,306.46",28.4,"1,701.12",20.9,"1,130.86",13.9,827.68,10.2
12,Algeria,2000,"1,11,022.59","87,595.60",78.9,"12,789.98",11.5,"5,463.83",4.9,"6,534.62",5.9,"11,428.54",10.3
24,Angola,2005,"61,610.76","37,732.06",61.2,...,...,352.00,0.6,"22,575.40",36.6,951.30,1.5
28,Antigua and Barbuda,2000,597.75,372.72,62.4,182.63,30.6,...,...,104.33,17.5,120.70,20.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Venezuela (Bolivarian Republic of),1999,"1,92,192.24","1,43,561.20",74.7,"33,991.00",17.7,"9,206.20",4.8,"32,959.70",17.1,"6,465.14",3.4
704,Viet Nam,2013,"2,78,441.86","1,47,703.33",53.0,"29,680.72",10.7,"31,767.39",11.4,"81,166.04",29.2,"17,805.10",6.4
887,Yemen,2012,"37,942.87","23,549.08",62.1,"5,739.01",15.1,"1,398.00",3.7,"10,879.79",28.7,"2,116.00",5.6


In [46]:
ghg.isnull().sum()

Country                                                                          0
Latest Year Available                                                            0
Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent)               0
GHG from energy (1000 tonnes of CO₂ equivalent)                                  0
GHG from energy, as percentage to total                                          0
GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)        0
GHG from energy, of which: from transport, as percentage to total                0
GHG from industrial processes and product use (1000 tonnes of CO₂ equivalent)    0
GHG from industrial processes and product use, as percentage to total            0
GHG from agriculture (1000 tonnes of CO₂ equivalent)                             0
GHG from agriculture, as percentage to total                                     0
GHG from waste (1000 tonnes of CO₂ equivalent)                                   0
GHG 

In [48]:
ghg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 192 entries, 4 to 716
Data columns (total 13 columns):
 #   Column                                                                         Non-Null Count  Dtype 
---  ------                                                                         --------------  ----- 
 0   Country                                                                        192 non-null    object
 1   Latest Year Available                                                          192 non-null    int64 
 2   Total GHG emissions without LULUCF (1000 tonnes of CO₂ equivalent)             192 non-null    object
 3   GHG from energy (1000 tonnes of CO₂ equivalent)                                192 non-null    object
 4   GHG from energy, as percentage to total                                        192 non-null    object
 5   GHG from energy, of which: from transport (1000 tonnes of CO₂ equivalent)      192 non-null    object
 6   GHG from energy, of which: from transpo

In [103]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"2010-01-01","end":"2019-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_switzerland=weather_df
#weather_df_switzerland

In [101]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"2010-01-01","end":"2019-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ireland=weather_df
#weather_df_ireland

In [99]:
url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set stations
stations = ["03969",
            "03962",
            "03955",
            "06660",
            "06700",
            "06601"
            ]

# Set API key (defined already)
headers = {
	"X-RapidAPI-Key": "38c2ec0004msh8992e15e6b32155p15fd43jsn0376b99a111e",
	"X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])



# Loop through all locations
for station in stations:
    # API call start for open weather
    querystring = {"station":station,"start":"2010-01-01","end":"2019-01-01"}

    response = requests.get(url, headers=headers, params=querystring)
    #API call end

    # time.sleep(1) #uncomment if you run into a query limit
    
    # Decode repsonse with json decoder
    weather_temp = response.json()
    
    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                        meta=["station"], 
                                        record_prefix="weather_",
                                        errors='ignore'
                                        )
    weather_temp_df['station']=station
    # concatenate dataframes
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df_ir_sw=weather_df
weather_df_ir_sw.replace(['03969', '03962','03955'], 'Ireland', inplace=True)
weather_df_ir_sw.replace(['06660', '06700', '06601'], 'Switzerland', inplace=True)
weather_df_ir_sw

,weather_date,weather_tavg,weather_tmin,weather_tmax,weather_prcp,weather_snow,weather_wdir,weather_wspd,weather_wpgt,weather_pres,weather_tsun,station
0,2010-01-01,-1.6,-3.3,-1.0,2.8,10.0,283.0,15.2,NaN,1010.7,None,Ireland
1,2010-01-02,-1.0,-3.4,3.5,0.0,10.0,267.0,20.7,NaN,1018.0,None,Ireland
2,2010-01-03,1.3,-2.4,3.3,0.3,10.0,85.0,19.5,NaN,1024.6,None,Ireland
3,2010-01-04,-0.9,-3.6,0.7,0.0,NaN,267.0,17.9,NaN,1020.9,None,Ireland
4,2010-01-05,-1.2,-3.6,2.3,NaN,NaN,302.0,20.3,NaN,1007.8,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
19723,2018-12-28,1.3,-1.3,4.0,0.0,0.0,57.0,4.1,14.8,1031.2,None,Switzerland
19724,2018-12-29,0.7,-0.6,1.4,0.0,0.0,335.0,3.8,13.0,1035.9,None,Switzerland
19725,2018-12-30,3.3,1.4,4.8,0.0,0.0,208.0,4.1,14.8,1036.6,None,Switzerland
19726,2018-12-31,5.4,3.3,7.7,0.0,0.0,136.0,3.2,13.0,1035.8,None,Switzerland


In [105]:
weather_df_ir_sw.drop(['weather_wpgt', 'weather_pres','weather_tsun'], axis='columns',inplace= True)

In [106]:
# the dataset should be uploaded to the schema 'capstone_hydrogenious'


# Import get_engine function from sql_functions_anja.py and set it to a variable called engine

from sql_functions_anja import get_engine
engine=get_engine()

# Import psycopg2
#import psycopg2

if engine!=None:
    try:
        weather_df_ir_sw.to_sql(name='weather_ir_sw', # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema='capstone_hydrogenious', # your class schema
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
    else:
        print('no engine')

The table was imported successfully.
no engine


In [67]:
from sql_functions_anja import get_dataframe

sql_query=f'select * from capstone_hydrogenious.crops_all_data;'
df_crops_all=get_dataframe(sql_query)

In [68]:
df_crops_all

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,143,Morocco,101,Canary seed,5312,Area harvested,2002,2002,ha,0.0,F
1,143,Morocco,101,Canary seed,5312,Area harvested,2003,2003,ha,0.0,F
2,143,Morocco,101,Canary seed,5312,Area harvested,2004,2004,ha,0.0,F
3,143,Morocco,101,Canary seed,5312,Area harvested,2005,2005,ha,0.0,F
4,143,Morocco,101,Canary seed,5312,Area harvested,2006,2006,ha,0.0,F
...,...,...,...,...,...,...,...,...,...,...,...
2513863,143,Morocco,101,Canary seed,5312,Area harvested,1997,1997,ha,0.0,None
2513864,143,Morocco,101,Canary seed,5312,Area harvested,1998,1998,ha,0.0,F
2513865,143,Morocco,101,Canary seed,5312,Area harvested,1999,1999,ha,0.0,F
2513866,143,Morocco,101,Canary seed,5312,Area harvested,2000,2000,ha,0.0,F


In [69]:
df_crops_all.isnull().sum()

Area Code            0
Area                 0
Item Code            0
Item                 0
Element Code         0
Element              0
Year Code            0
Year                 0
Unit                 0
Value           154312
Flag            530383
dtype: int64

In [71]:
df_crops_ir_sw = df_crops_all.query("(`Area` == 'Ireland') or (`Area` == 'Switzerland')")
df_crops_ir_sw

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
458644,211,Switzerland,515,Apples,5312,Area harvested,1961,1961,ha,2700.0,F
458645,211,Switzerland,515,Apples,5312,Area harvested,1962,1962,ha,4800.0,F
458646,211,Switzerland,515,Apples,5312,Area harvested,1963,1963,ha,3600.0,F
458647,211,Switzerland,515,Apples,5312,Area harvested,1964,1964,ha,4000.0,F
458648,211,Switzerland,515,Apples,5312,Area harvested,1965,1965,ha,2478.0,None
...,...,...,...,...,...,...,...,...,...,...,...
2250910,104,Ireland,1735,Vegetables Primary,5510,Production,2015,2015,tonnes,222293.0,A
2250911,104,Ireland,1735,Vegetables Primary,5510,Production,2016,2016,tonnes,231989.0,A
2250912,104,Ireland,1735,Vegetables Primary,5510,Production,2017,2017,tonnes,204672.0,A
2250913,104,Ireland,1735,Vegetables Primary,5510,Production,2018,2018,tonnes,170700.0,A
